# 1. Install Dependencies

In [1]:
# Install required libraries
!pip install -U datasets transformers evaluate peft bitsandbytes
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


# 2. Preprocess Data

In [2]:
# Load data
from datasets import load_dataset
imdb = load_dataset("imdb")
print(imdb)

train_dataset = imdb['train'].shuffle(seed=42).select(range(500))
test_dataset = imdb['test'].shuffle(seed=42).select(range(500))

print(len(train_dataset))
print(len(test_dataset))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})
500
500


In [35]:
from datasets import load_dataset
import torch
import transformers
from transformers import Trainer, TrainingArguments
from transformers.models.auto import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
from transformers import DataCollatorWithPadding
import evaluate

transformers.set_seed(42)
torch.manual_seed(42)

CHECKPOINT = "google-bert/bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)




Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [5]:
config = AutoConfig.from_pretrained(CHECKPOINT, num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, config=config)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

static_memory = model.get_memory_footprint()/ (2 ** 20)
print(f"Static memory footprint: {static_memory:.2f} MB")



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Static memory footprint: 417.66 MB


In [6]:
best_lr = 3e-5
best_batch_size = 16
best_epoch = 3
best_weight_decay = 0.1

# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}


training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=best_lr,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    weight_decay=best_weight_decay,
    num_train_epochs=best_epoch
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Reset peak tracking
torch.cuda.reset_peak_memory_stats()

# Train the model
train_results = trainer.train()
print(train_results)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)


Step,Training Loss


TrainOutput(global_step=96, training_loss=0.3531537453333537, metrics={'train_runtime': 32.5667, 'train_samples_per_second': 46.059, 'train_steps_per_second': 2.948, 'total_flos': 390787750528560.0, 'train_loss': 0.3531537453333537, 'epoch': 3.0})


{'eval_loss': 0.31242674589157104, 'eval_accuracy': 0.878, 'eval_f1': 0.8767676767676768, 'eval_recall': 0.8821138211382114, 'eval_precision': 0.8714859437751004, 'eval_runtime': 3.301, 'eval_samples_per_second': 151.471, 'eval_steps_per_second': 9.694, 'epoch': 3.0}


In [7]:
peak_memory = torch.cuda.max_memory_allocated() / (2 ** 20)
print(f"Peak memory usage: {peak_memory:.2f} MB")

Peak memory usage: 7923.53 MB


#3. LoRA (lr: 3e-5)

In [8]:
from peft import LoraConfig, get_peft_model

config = AutoConfig.from_pretrained(CHECKPOINT, num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, config=config)


lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "key", "value"],
    modules_to_save=["classifier"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS",
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 443,906 || all params: 109,927,684 || trainable%: 0.4038


In [9]:
static_memory = model.get_memory_footprint()/ (2 ** 20)
print(f"Static memory footprint: {static_memory:.2f} MB")


Static memory footprint: 419.35 MB


In [18]:
from transformers import DataCollatorWithPadding


# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

best_lr = 3e-5
best_batch_size = 16
best_epoch = 3
best_weight_decay = 0.1

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=best_lr,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_epoch,
    weight_decay=best_weight_decay,
    label_names=["labels"],
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Reset peak tracking
torch.cuda.reset_peak_memory_stats()

# Train the model
train_results = trainer.train()
print(train_results)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

Step,Training Loss


TrainOutput(global_step=96, training_loss=0.6942718823750814, metrics={'train_runtime': 23.4305, 'train_samples_per_second': 64.019, 'train_steps_per_second': 4.097, 'total_flos': 392813165761632.0, 'train_loss': 0.6942718823750814, 'epoch': 3.0})


{'eval_loss': 0.6939959526062012, 'eval_accuracy': 0.514, 'eval_f1': 0.549165120593692, 'eval_recall': 0.6016260162601627, 'eval_precision': 0.5051194539249146, 'eval_runtime': 3.4138, 'eval_samples_per_second': 146.463, 'eval_steps_per_second': 9.374, 'epoch': 3.0}


In [19]:
peak_memory = torch.cuda.max_memory_allocated() / (2 ** 20)
print(f"Peak memory usage: {peak_memory:.2f} MB")

Peak memory usage: 4772.76 MB


#4. LoRA (lr: 1e-3)

In [11]:
from peft import LoraConfig, get_peft_model

config = AutoConfig.from_pretrained(CHECKPOINT, num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, config=config)


lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "key", "value"],
    modules_to_save=["classifier"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS",
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 443,906 || all params: 109,927,684 || trainable%: 0.4038


In [ ]:
from transformers import DataCollatorWithPadding

# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

best_lr = 1e-3
best_batch_size = 16
best_epoch = 3
best_weight_decay = 0.1

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=best_lr,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_epoch,
    weight_decay=best_weight_decay,
    label_names=["labels"],
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Reset peak tracking
torch.cuda.reset_peak_memory_stats()

# Train the model
train_results = trainer.train()
print(train_results)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

Step,Training Loss


TrainOutput(global_step=32, training_loss=0.7014949321746826, metrics={'train_runtime': 10.9754, 'train_samples_per_second': 91.113, 'train_steps_per_second': 2.916, 'total_flos': 266293751808000.0, 'train_loss': 0.7014949321746826, 'epoch': 2.0})


{'eval_loss': 0.6224043369293213, 'eval_accuracy': 0.84, 'eval_f1': 0.8380566801619433, 'eval_recall': 0.8414634146341463, 'eval_precision': 0.8346774193548387, 'eval_runtime': 2.2531, 'eval_samples_per_second': 221.918, 'eval_steps_per_second': 7.101, 'epoch': 2.0}


In [20]:
peak_memory = torch.cuda.max_memory_allocated() / (2 ** 20)
print(f"Peak memory usage: {peak_memory:.2f} MB")

Peak memory usage: 4772.76 MB


# 5. QLoRA (lr: 3e-5)

In [12]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import BitsAndBytesConfig


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

config = AutoConfig.from_pretrained(CHECKPOINT, num_labels=2)

model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, config=config, quantization_config=bnb_config, device_map="auto")
model = prepare_model_for_kbit_training(model)

model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear4bit(in_features=768, out_features=768, bias=True)
              (key): Linear4bit(in_features=768, out_features=768, bias=True)
              (value): Linear4bit(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear4bit(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [13]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "key", "value"],
    modules_to_save=["classifier"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS",
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

trainable params: 443,906 || all params: 109,927,684 || trainable%: 0.4038


In [14]:
static_memory = model.get_memory_footprint()/ (2 ** 20)
print(f"Static memory footprint: {static_memory:.2f} MB")

Static memory footprint: 133.88 MB


In [15]:
from transformers import DataCollatorWithPadding

# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

best_lr = 3e-5
best_batch_size = 16
best_epoch = 3
best_weight_decay = 0.1

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=best_lr,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_epoch,
    weight_decay=best_weight_decay,
    label_names=["labels"],
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Reset peak tracking
torch.cuda.reset_peak_memory_stats()

# Train the model
train_results = trainer.train()
print(train_results)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


TrainOutput(global_step=96, training_loss=0.6862141291300455, metrics={'train_runtime': 34.1203, 'train_samples_per_second': 43.962, 'train_steps_per_second': 2.814, 'total_flos': 392813165761632.0, 'train_loss': 0.6862141291300455, 'epoch': 3.0})


{'eval_loss': 0.6823294758796692, 'eval_accuracy': 0.606, 'eval_f1': 0.55125284738041, 'eval_recall': 0.491869918699187, 'eval_precision': 0.6269430051813472, 'eval_runtime': 3.5259, 'eval_samples_per_second': 141.807, 'eval_steps_per_second': 9.076, 'epoch': 3.0}


In [16]:
peak_memory = torch.cuda.max_memory_allocated() / (2 ** 20)
print(f"Peak memory usage: {peak_memory:.2f} MB")

Peak memory usage: 3089.76 MB


#6. QLoRA (lr: 1e-3)

In [36]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import BitsAndBytesConfig


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

config = AutoConfig.from_pretrained(CHECKPOINT, num_labels=2)

model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, config=config, quantization_config=bnb_config, device_map="auto")
model = prepare_model_for_kbit_training(model)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "key", "value"],
    modules_to_save=["classifier"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS",
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

trainable params: 443,906 || all params: 109,927,684 || trainable%: 0.4038


In [38]:
from transformers import DataCollatorWithPadding

# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

best_lr = 1e-3
best_batch_size = 16
best_epoch = 3
best_weight_decay = 0.1

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=best_lr,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_epoch,
    weight_decay=best_weight_decay,
    label_names=["labels"],
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Reset peak tracking
torch.cuda.reset_peak_memory_stats()

# Train the model
train_results = trainer.train()
print(train_results)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


TrainOutput(global_step=96, training_loss=0.5291921695073446, metrics={'train_runtime': 33.9911, 'train_samples_per_second': 44.129, 'train_steps_per_second': 2.824, 'total_flos': 392813165761632.0, 'train_loss': 0.5291921695073446, 'epoch': 3.0})


{'eval_loss': 0.3699442744255066, 'eval_accuracy': 0.86, 'eval_f1': 0.8638132295719845, 'eval_recall': 0.9024390243902439, 'eval_precision': 0.8283582089552238, 'eval_runtime': 3.5303, 'eval_samples_per_second': 141.631, 'eval_steps_per_second': 9.064, 'epoch': 3.0}


In [21]:
peak_memory = torch.cuda.max_memory_allocated() / (2 ** 20)
print(f"Peak memory usage: {peak_memory:.2f} MB")

Peak memory usage: 1372.62 MB


# 7. Prompt Tuning (lr: 3e-5)


In [40]:
from datasets import load_dataset
import torch
from transformers import Trainer, TrainingArguments
from transformers.models.auto import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import transformers
import evaluate


torch.manual_seed(42)
transformers.set_seed(42)

CHECKPOINT = "google-bert/bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=492)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [30]:
from peft import PromptTuningConfig, get_peft_model

config = AutoConfig.from_pretrained(CHECKPOINT, num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, config=config)

prompt_config = PromptTuningConfig(
    task_type="SEQ_CLS",
    num_virtual_tokens=20,
    token_dim=768,
    num_transformer_submodules=1,
    num_attention_heads=12,
    num_layers=12,
    tokenizer_name_or_path=CHECKPOINT,
)

prompt_config.modules_to_save = ["classifier", "pooler"]

model = get_peft_model(model, prompt_config)
model.print_trainable_parameters()



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 607,490 || all params: 110,091,268 || trainable%: 0.5518


In [27]:
static_memory = model.get_memory_footprint()/ (2 ** 20)
print(f"Static memory footprint: {static_memory:.2f} MB")

Static memory footprint: 417.66 MB


In [31]:
from transformers import DataCollatorWithPadding

# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

best_lr = 3e-5
best_batch_size = 16
best_epoch = 3
best_weight_decay = 0.1


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=best_lr,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_epoch,
    weight_decay=best_weight_decay,
    label_names=["labels"],
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Reset peak tracking
torch.cuda.reset_peak_memory_stats()

# Train the model
train_results = trainer.train()
print(train_results)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

Step,Training Loss


TrainOutput(global_step=96, training_loss=0.68230668703715, metrics={'train_runtime': 21.2851, 'train_samples_per_second': 70.472, 'train_steps_per_second': 4.51, 'total_flos': 378711338231520.0, 'train_loss': 0.68230668703715, 'epoch': 3.0})


{'eval_loss': 0.6792780756950378, 'eval_accuracy': 0.604, 'eval_f1': 0.6162790697674418, 'eval_recall': 0.6463414634146342, 'eval_precision': 0.5888888888888889, 'eval_runtime': 3.2629, 'eval_samples_per_second': 153.236, 'eval_steps_per_second': 9.807, 'epoch': 3.0}


# 8. Prompt Tuning (lr: 1e-3)

In [44]:
from peft import PromptTuningConfig, get_peft_model

config = AutoConfig.from_pretrained(CHECKPOINT, num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, config=config)

prompt_config = PromptTuningConfig(
    task_type="SEQ_CLS",
    num_virtual_tokens=20,
    token_dim=768,
    num_transformer_submodules=1,
    num_attention_heads=12,
    num_layers=12,
    tokenizer_name_or_path=CHECKPOINT,
)

prompt_config.modules_to_save = ["classifier", "pooler"]

model = get_peft_model(model, prompt_config)
model.print_trainable_parameters()



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 607,490 || all params: 110,091,268 || trainable%: 0.5518


In [45]:
from transformers import EarlyStoppingCallback
from transformers import DataCollatorWithPadding

# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}

early_stopping = EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.0)

batch_size = 32
lr = 1e-3
epoch = 15
weight_decay = 0.1

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay=weight_decay,
    load_best_model_at_end=True,
    label_names=["labels"],
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

# Reset peak tracking
torch.cuda.reset_peak_memory_stats()

# Train the model
train_results = trainer.train()
print(train_results)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Recall,Precision
1,No log,0.647470,0.628000,0.539604,0.443089,0.689873
2,No log,0.635192,0.666000,0.651357,0.634146,0.669528
3,No log,0.642899,0.642000,0.532637,0.414634,0.744526
4,No log,0.618991,0.686000,0.630588,0.544715,0.748603
5,No log,0.595554,0.688000,0.663793,0.626016,0.706422
6,No log,0.600584,0.708000,0.691983,0.666667,0.719298
7,No log,0.603944,0.704000,0.674009,0.621951,0.735577


TrainOutput(global_step=112, training_loss=0.5822361537388393, metrics={'train_runtime': 70.1581, 'train_samples_per_second': 106.901, 'train_steps_per_second': 3.421, 'total_flos': 891034366320000.0, 'train_loss': 0.5822361537388393, 'epoch': 7.0})


{'eval_loss': 0.5955542922019958, 'eval_accuracy': 0.688, 'eval_f1': 0.6637931034482759, 'eval_recall': 0.6260162601626016, 'eval_precision': 0.7064220183486238, 'eval_runtime': 3.1703, 'eval_samples_per_second': 157.716, 'eval_steps_per_second': 5.047, 'epoch': 7.0}


In [46]:
peak_memory = torch.cuda.max_memory_allocated() / (2 ** 20)
print(f"Peak memory usage: {peak_memory:.2f} MB")

Peak memory usage: 7407.76 MB
